In [ ]:
import pandas as pd
from pandas import DataFrame
import portfolio

In [ ]:
def calculate_sma(df: DataFrame, window):
    return df['Close'].rolling(window=window).mean()

In [ ]:
# Read in the pre-processed data
ticker = "AAPL"
data_path = f"../data/{ticker}_processed_hourly_data.csv"
data = pd.read_csv(data_path, index_col='Datetime', parse_dates=True)

In [ ]:
display(data)

In [ ]:
# Parameters
short_window = 10
long_window = 100

# Time period (for days set it to 252 and for hours to 1638)
time_period = 1638

# Calculate the short and long-term SMA's
data['SMA_short'] = calculate_sma(data, short_window)
data['SMA_long'] = calculate_sma(data, long_window)

# Filter out the rows that will be null
data = data[data['SMA_long'].notna()]

# Run the SMA strategy
portfolio = portfolio.Portfolio(initial_cash=100000)
holding = False

stop_loss_percentage = 0.02
take_profit_percentage = 0.05

In [ ]:
for index, row in data.iterrows():
    # Add some stop loss and take profit logic here
    current_price = row['Close']
    if portfolio.holdings > 0 and portfolio.last_buy_price:

        if current_price < (portfolio.last_buy_price * (1 - stop_loss_percentage)):
            portfolio.sell(current_price, portfolio.holdings)
            holding = False
        elif current_price > (portfolio.last_buy_price * (1 + take_profit_percentage)):
            portfolio.sell(current_price, portfolio.holdings)
            holding = False

    if row['SMA_short'] > row['SMA_long'] and not holding:
        portfolio.buy(row['Close'], 250)
        holding = True
    elif row['SMA_short'] < row['SMA_long'] and holding:
        portfolio.sell(row['Close'], 250)
        holding = False

# Get the final value at the last day of the data
final_value = portfolio.get_value(data.iloc[-1]['Close'])
print(f"Final Portfolio Value = {final_value}")

In [ ]:
portfolio.trade_summary()